In [112]:
#Algorithm to calculate the jaccard similarity, minhash and local senstive hanshing. 
import pandas as pd
import binascii
import random


# load airline reviews
reviews_all = pd.read_csv("data/skytrax-reviews-dataset-master/data/airline.csv").content

# num of reviews to analyse
numReviews = 1000

# subset of reviews
reviews = reviews_all[0:numReviews]

del reviews_all



In [113]:
################
# Shingling
################

# set shingle character lenght
k_shingle = 10

allShingleSets = {}


for index, review in enumerate(reviews):
    # Contain all unique (no duplicate) shingles of the review
    shingleSet = set()
    for i in range(len(review) - k_shingle + 1):
        shingle = review[i: i + k_shingle].encode()
        
        # hashing to 32-bit integer
        crc = binascii.crc32(shingle) & 0xffffffff
        # add hash value to document shingleset if not present yet
        shingleSet.add(crc)
    
    # store all shingle sets from each review together
    allShingleSets[index] = shingleSet

In [114]:
#####################
# Jaccard Similarity
#####################
jaccardSimilarities = {}
for i, shingleSet1 in allShingleSets.items():
    set1 = shingleSet1
    for j in range(i+1, len(allShingleSets)):
        set2 = allShingleSets[j]
    
        # calculate and store jaccard similarities
        jaccardSimilarities[(i, j)] = (len(set1.intersection(set2))/len(set1.union(set2)))

In [115]:

for index, value in jaccardSimilarities.items():
    if value >= 0.4:
        print(index, value)
        print(reviews[index[0]])
        print("--------")
        print(reviews[index[1]])
        print()

(222, 236) 1.0
I flew from Chicago O'Hare to Dublin and from Dublin to Amsterdam and Amsterdam back to Dublin and Dublin back to O'Hare and I must say I was pleased with the airline. The food was good the entertainment was good and the leg room on the flights was great. Only issue I did have was from Amsterdam to Dublin because we arrived 30 minutes left and nearly missed our connecting flight from Dublin to O'Hare however we made it on time with all of our belongings. Will definitely use Aer Lingus again on any future trips to Ireland.
--------
I flew from Chicago O'Hare to Dublin and from Dublin to Amsterdam and Amsterdam back to Dublin and Dublin back to O'Hare and I must say I was pleased with the airline. The food was good the entertainment was good and the leg room on the flights was great. Only issue I did have was from Amsterdam to Dublin because we arrived 30 minutes left and nearly missed our connecting flight from Dublin to O'Hare however we made it on time with all of our b

In [116]:
################
# MinHash
################
numHashes = 100
maxShingle = 2**32-1
maxShingle

maxPrime = 4294967311

# hash function takes form h(x) = (a*x + b) mod c
# a anb b are random coefficients generated by function below, 
# c is equal to the first prime number outside our hash bounds
def randomCoefficients(numHashes):
    # list all coefficients
    coeffList = []

    for i in range(0, numHashes):
        randCoeff = random.randint(0, maxShingle)

        # ensure unique coefficients
        while randCoeff in coeffList:
            randCoeff = random.randint(0, maxShingle)

        coeffList.append(randCoeff)
    return coeffList

coeffA = randomCoefficients(numHashes)
coeffB = randomCoefficients(numHashes)

# signature vectors
signatures = []

# hashfunction on shingle and take lowest value
for index, shingleSet in allShingleSets.items():
    
    sig = []
    
    for i in range(0, numHashes):
        minHash = maxPrime + 1
        for shingle in shingleSet:
            hashCode = (coeffA[i]*shingle + coeffB[i]) % maxPrime
            
            if hashCode < minHash:
                minHash = hashCode
        
        sig.append(minHash)
    
    signatures.append(sig)

    

In [117]:
#####################
# Compare signatures
#####################

est_jacc = {}

# evaluate each signature with another
for i in range(0, numReviews):
    sig1 = signatures[i]
    
    for j in range (i+1, numReviews):
        sig2 = signatures[j]
        
        # count the number of equivalent values in signature
        count = 0
        
        for k in range(0, numHashes):
            if sig1[k] == sig2[k]:
                count += 1
        
        # calculate ratio of matched values in signatures
        est_jacc[(i, j)] = count/numHashes
                
        

In [118]:
for index, value in est_jacc.items():
    if value >= 0.4:
        print(index, value)
        print(reviews[index[0]])
        print("--------")
        print(reviews[index[1]])
        print()

(222, 236) 1.0
I flew from Chicago O'Hare to Dublin and from Dublin to Amsterdam and Amsterdam back to Dublin and Dublin back to O'Hare and I must say I was pleased with the airline. The food was good the entertainment was good and the leg room on the flights was great. Only issue I did have was from Amsterdam to Dublin because we arrived 30 minutes left and nearly missed our connecting flight from Dublin to O'Hare however we made it on time with all of our belongings. Will definitely use Aer Lingus again on any future trips to Ireland.
--------
I flew from Chicago O'Hare to Dublin and from Dublin to Amsterdam and Amsterdam back to Dublin and Dublin back to O'Hare and I must say I was pleased with the airline. The food was good the entertainment was good and the leg room on the flights was great. Only issue I did have was from Amsterdam to Dublin because we arrived 30 minutes left and nearly missed our connecting flight from Dublin to O'Hare however we made it on time with all of our b

In [119]:
#########
# LSH
#########
bands = 50
rows = 2
#Approximate threshold
threshold = (1/bands)**(1/rows)
print(threshold)

candidate_pairs = {}

# for b1 in range(0, bands):
#     for column in signatures:
#         band_set1 = set(column[b1*rows:b1*rows+rows])
#         for b2 in range(b1+1, bands):
#             band_set2 = set(column[b2*rows:b2*rows+rows])
# #             jaccard_sim_bands = (len(band_set1.intersection(band_set2))/len(band_set1.union(band_set2)))
#             print(jaccard_sim_bands)
#             if jaccard_sim_bands >= threshold:
#                 candidatepairs.append(jaccard_sim_band)
#                 print(candidatepairs)
#                 print(column)

                
for band in range(0, bands):
    for i in range(0, numReviews):
        band_hash1 = hash(tuple(signatures[i][band*rows:band*rows+rows]))
        for j in range(i+1, numReviews):
            band_hash2 = hash(tuple(signatures[j][band*rows:band*rows+rows]))
            if band_hash1 == band_hash2:
                c = (i, j)
                if c not in candidate_pairs:
                    candidate_pairs[(i, j)] = 1
                else:
                    candidate_pairs[(i, j)] += 1
                
for c_pair, matching_bands in candidate_pairs.items() :
    if (matching_bands/bands) >= threshold:
        print(c_pair)
        print(matching_bands)
        sig1 = signatures[c_pair[0]]
        sig2 = signatures[c_pair[1]]
        
         # count the number of equivalent values in signature
        count = 0
        
        for k in range(0, numHashes):
            if sig1[k] == sig2[k]:
                count += 1
                
        # calculate ratio of matched values in signatures
        candidates_similarity = count/numHashes
        print(candidates_similarity)
        
        print(reviews[c_pair[0]])
        print("-----")
        print(reviews[c_pair[1]])
        
    if matching_bands > 2:
        print(c_pair)
        
                
#         for i in range(0, rows):
#             print(len(band))
#             set1 = set(band[i])
#             for j in range(i+1, rows):
#                 set2 = set(band[j])
#                 jaccard_sim_band = (len(set1.intersection(set2))/len(set1.union(set2)))
#                 print(jaccard_sim_band)
#                 if jaccard_sim_band >= threshold:
#                     candidatepairs.append(jaccard_sim_band)
#                     print(candidatepairs)

        
# lsh = []
# for band in range(0, bands):
#     lsh.append(hash(signatures[band*rows:band*rows+rows]))

0.1414213562373095
(222, 236)
50
1.0
I flew from Chicago O'Hare to Dublin and from Dublin to Amsterdam and Amsterdam back to Dublin and Dublin back to O'Hare and I must say I was pleased with the airline. The food was good the entertainment was good and the leg room on the flights was great. Only issue I did have was from Amsterdam to Dublin because we arrived 30 minutes left and nearly missed our connecting flight from Dublin to O'Hare however we made it on time with all of our belongings. Will definitely use Aer Lingus again on any future trips to Ireland.
-----
I flew from Chicago O'Hare to Dublin and from Dublin to Amsterdam and Amsterdam back to Dublin and Dublin back to O'Hare and I must say I was pleased with the airline. The food was good the entertainment was good and the leg room on the flights was great. Only issue I did have was from Amsterdam to Dublin because we arrived 30 minutes left and nearly missed our connecting flight from Dublin to O'Hare however we made it on tim